In [1]:
# !pip install transformers
# !pip install vocab
# !pip install pyvi
# !pip install torch_geometric
# !pip install torch==1.8.1+cu101 torchvision==0.9.1+cu101 torchaudio==0.8.1 -f https://download.pytorch.org/whl/torch_stable.html
# !pip install torch-scatter
# !pip install tensorboardX

In [2]:
import torch
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data.sampler import SubsetRandomSampler
import torch.optim as optim
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
from torch import nn
from torch_geometric.utils import softmax
from torch_scatter import scatter_add

import pickle, pandas as pd
import numpy as np
import os
import argparse
import random
import time
from collections import Counter

from sklearn import metrics
from sklearn.metrics import f1_score, accuracy_score

from pyvi.ViTokenizer import tokenize

from transformers import AutoTokenizer, AutoModel

In [3]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
def seed_everything(seed=2021):
    print(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

In [5]:
seed_everything()

2021


# Load data

In [6]:
no_label_data = 9
data_path = f"../DataPreprocess/Cleaned_Data/{no_label_data}_label/clean_data.csv"
train_data_path = f"../DataPreprocess/Cleaned_Data/{no_label_data}_label/train_data.csv"
dev_data_path = f"../DataPreprocess/Cleaned_Data/{no_label_data}_label/dev_data.csv"
test_data_path = f"../DataPreprocess/Cleaned_Data/{no_label_data}_label/test_data.csv"

In [7]:
train_data = pd.read_csv(train_data_path).fillna("")
valid_data = pd.read_csv(dev_data_path).fillna("")
test_data = pd.read_csv(test_data_path).fillna("")
df = pd.read_csv(data_path).fillna("")

print(f"Train Set Shape: {train_data.shape}")
print(f"Valid Set Shape: {valid_data.shape}")
print(f"Test Set Shape: {test_data.shape}")
print(f"Full Set Shape: {df.shape}")

df.head(5)

Train Set Shape: (10888, 12)
Valid Set Shape: (3077, 12)
Test Set Shape: (1568, 12)
Full Set Shape: (15533, 12)


,Index,Utterance,Speaker,Id_speaker,Utterance_id,Date,Time,Emotion,Emotion_Mutiple,Dialog_id,Label,Utterance_clean
0,1,Bao tiền,Nguyễn Thanh Tú,100031059109987,1,18/02/2022,08:07:47,Neutral,Neutral,1,0,Bao tiền
1,2,Nguyễn Thanh Tú bạn có khum haha,Nguyễn Thị Diễm,100007602498241,2,18/02/2022,08:08:10,Joy,Joy,1,1,bạn có khum haha
2,3,Nguyễn Thị Diễm nổ giá đii đừng ib.,Nguyễn Thanh Tú,100031059109987,3,18/02/2022,08:08:27,Anger,Anger,1,4,nổ giá đii đừng ib .
3,4,T có nha,Dao Phuong Anh,100009157681703,1,18/02/2022,08:37:06,Neutral,Neutral,2,0,T có nha
4,5,Dao Phuong Anh check ib ạ,Nguyễn Thị Diễm,100007602498241,2,18/02/2022,08:37:18,Neutral,Neutral,2,0,check ib ạ


# Dataloader

In [8]:
class ERCDataset(Dataset):

    def __init__(self, path=None, n_classes=9, train=True, dev=True):
        self.dialogIDs, self.dialogSpeakers, self.dialogLabels, self.dialogText,\
        self.dialogSentence, self.trainIds, self.testIds, self.devIds = pickle.load(open(path, 'rb'))

        '''
        label index mapping = {'frustration': 0, 'fear': 1, 'anger': 2, 'disgust': 3, 'sadness' : 4, 'joy': 5, 'surprise': 6, 'excited': 7, 'neutral': 8}
        '''

        self.keys = []
        if train:
            for x in self.trainIds:
                self.keys.append(x)
        elif dev:
            for x in self.devIds:
                self.keys.append(x)
        else:
            for x in self.testIds:
                self.keys.append(x)   
                
        self.len = len(self.keys)

    def calculate_weight(self):
        class_count = Counter()

        for vid in self.trainIds:
            for uid in self.dialogLabels[vid]:
                class_count[uid] += 1

        total = sum(class_count.values())
        class_weights = {key: total / val for key, val in class_count.items()}
        return list(class_weights.values())

    def __getitem__(self, index):
        idx = self.keys[index]
        return torch.FloatTensor(self.dialogText[idx]), \
               torch.FloatTensor(self.dialogSpeakers[idx]), \
               torch.FloatTensor([1] * len(self.dialogLabels[idx])), \
               torch.LongTensor(self.dialogLabels[idx]), \
               idx

    def __len__(self):
        return self.len

    def collate_fn(self, data):
        dat = pd.DataFrame(data)
        return [pad_sequence(dat[i]) if i < 2 else pad_sequence(dat[i], True) if i < 4 else dat[i].tolist() for i in dat]


# Loss function

In [9]:
class FocalLoss(nn.Module):
    def __init__(self, gamma=0, alpha=None, size_average=True):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.alpha = alpha
        if isinstance(alpha, (float, int)): self.alpha = torch.Tensor([alpha, 1 - alpha])
        if isinstance(alpha, list): self.alpha = torch.Tensor(alpha)
        self.size_average = size_average

    def forward(self, input, target):
        if input.dim() > 2:
            input = input.view(input.size(0), input.size(1), -1)
            input = input.transpose(1, 2)
            input = input.contiguous().view(-1, input.size(2))
        target = target.view(-1, 1)

        logpt = input.gather(1, target).view(-1)
        pt = Variable(logpt.data.exp())

        if self.alpha is not None:
            if self.alpha.type() != input.data.type():
                self.alpha = self.alpha.type_as(input.data)
            at = self.alpha.gather(0, target.data.view(-1))
            logpt = logpt * Variable(at)

        loss = -1 * (1 - pt) ** self.gamma * logpt
        if self.size_average:
            return loss.mean()
        else:
            return loss.sum()

#Model

In [10]:
def pad(tensor, length, cuda_flag):
    if isinstance(tensor, Variable):
        var = tensor
        if length > var.size(0):
            if cuda_flag:
                return torch.cat([var, torch.zeros(length - var.size(0), *var.size()[1:]).cuda()])
            else:
                return torch.cat([var, torch.zeros(length - var.size(0), *var.size()[1:])])
        else:
            return var
    else:
        if length > tensor.size(0):
            if cuda_flag:
                return torch.cat([tensor, torch.zeros(length - tensor.size(0), *tensor.size()[1:]).cuda()])
            else:
                return torch.cat([tensor, torch.zeros(length - tensor.size(0), *tensor.size()[1:])])
        else:
            return tensor


def feature_transfer(bank_s_, bank_p_, seq_lengths, cuda_flag=False):
    input_conversation_length = torch.tensor(seq_lengths)
    start_zero = input_conversation_length.data.new(1).zero_()
    if cuda_flag:
        input_conversation_length = input_conversation_length.cuda()
        start_zero = start_zero.cuda()

    max_len = max(seq_lengths)
    start = torch.cumsum(torch.cat((start_zero, input_conversation_length[:-1])), 0)
    # (l,b,h)
    bank_s = torch.stack(
        [pad(bank_s_.narrow(0, s, l), max_len, cuda_flag) for s, l in zip(start.data.tolist(), input_conversation_length.data.tolist())], 0
    ).transpose(0, 1)
    bank_p = torch.stack(
        [pad(bank_p_.narrow(0, s, l), max_len, cuda_flag) for s, l in zip(start.data.tolist(), input_conversation_length.data.tolist())], 0
    ).transpose(0, 1)

    return bank_s, bank_p

In [11]:
class ReasonModule(nn.Module):
    def __init__(self, in_channels=200, processing_steps=0, num_layers=1):
        """
        Reasoning Module
        """
        super(ReasonModule, self).__init__()

        self.in_channels = in_channels
        self.out_channels = 2 * in_channels
        self.processing_steps = processing_steps
        self.num_layers = num_layers
        if processing_steps > 0:
            self.lstm = nn.LSTM(self.out_channels, self.in_channels, num_layers)  # 400,200,1
            self.lstm.reset_parameters()

    def forward(self, x, batch, q_star):
        if self.processing_steps <= 0: return q_star

        batch_size = batch.max().item() + 1
        h = (x.new_zeros((self.num_layers, batch_size, self.in_channels)),
             x.new_zeros((self.num_layers, batch_size, self.in_channels)))
        for i in range(self.processing_steps):
            q, h = self.lstm(q_star.unsqueeze(0), h)
            q = q.view(batch_size, self.in_channels)
            e = (x * q[batch]).sum(dim=-1, keepdim=True)
            a = softmax(e, batch, num_nodes=batch_size)
            r = scatter_add(a * x, batch, dim=0, dim_size=batch_size)
            q_star = torch.cat([q, r], dim=-1)
        return q_star

    def __repr__(self):
        return '{}({}, {})'.format(self.__class__.__name__, self.in_channels, self.out_channels)

In [12]:
class CognitionNetwork(nn.Module):
    def __init__(self, n_features=200, n_classes=9, dropout=0.2, cuda_flag=False, reason_steps=None):
        """
        Multi-turn Reasoning Modules
        """
        super(CognitionNetwork, self).__init__()
        self.cuda_flag = cuda_flag
        self.fc = nn.Linear(n_features, n_features * 2)
        self.steps = reason_steps if reason_steps is not None else [0, 0]
        self.reason_modules = nn.ModuleList([
            ReasonModule(in_channels=n_features, processing_steps=self.steps[0], num_layers=1),
            ReasonModule(in_channels=n_features, processing_steps=self.steps[1], num_layers=1)
        ])
        self.dropout = nn.Dropout(dropout)
        self.smax_fc = nn.Linear(n_features * 4, n_classes)

    def forward(self, U_s, U_p, seq_lengths):
        # (b) <== (l,b,h)
        batch_size = U_s.size(1)
        batch_index, context_s_, context_p_ = [], [], []
        for j in range(batch_size):
            batch_index.extend([j] * seq_lengths[j])
            context_s_.append(U_s[:seq_lengths[j], j, :])
            context_p_.append(U_p[:seq_lengths[j], j, :])

        batch_index = torch.tensor(batch_index)
        bank_s_ = torch.cat(context_s_, dim=0)
        bank_p_ = torch.cat(context_p_, dim=0)
        if self.cuda_flag:
            batch_index = batch_index.cuda()
            bank_s_ = bank_s_.cuda()
            bank_p_ = bank_p_.cuda()

        # (l,b,h) << (l*b,h)
        bank_s, bank_p = feature_transfer(bank_s_, bank_p_, seq_lengths, self.cuda_flag)

        feature_ = []
        for t in range(bank_s.size(0)):
            # (2*h) <== (h)
            q_star = self.fc(bank_s[t])
            q_situ = self.reason_modules[0](bank_s_, batch_index, q_star)
            feature_.append(q_situ.unsqueeze(0))
        feature_s = torch.cat(feature_, dim=0)

        feature_ = []
        for t in range(bank_p.size(0)):
            q_star = self.fc(bank_p[t])
            q_party = self.reason_modules[1](bank_p_, batch_index, q_star)
            feature_.append(q_party.unsqueeze(0))
        feature_v = torch.cat(feature_, dim=0)

        # (l,b,2*2*h)
        hidden = torch.cat([feature_v, feature_s], dim=-1)
        hidden = self.dropout(F.relu(hidden))
        log_prob = F.log_softmax(self.smax_fc(hidden), 2)
        log_prob = torch.cat([log_prob[:, j, :][:seq_lengths[j]] for j in range(len(seq_lengths))])
        return log_prob

In [13]:
class DialogueCRN(nn.Module):
    def __init__(self, base_model='LSTM', base_layer=2, input_size=None, hidden_size=None, n_speakers=29,
                 n_classes=9, dropout=0.2, cuda_flag=False, reason_steps=None):
        """
        Contextual Reasoning Network
        """

        super(DialogueCRN, self).__init__()
        self.base_model = base_model
        self.n_speakers = n_speakers
        self.hidden_size = hidden_size

        if self.base_model == 'LSTM':
            self.rnn = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=base_layer, bidirectional=True, dropout=dropout)
            self.rnn_parties = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=base_layer, bidirectional=True, dropout=dropout)
        elif self.base_model == 'GRU':
            self.rnn = nn.GRU(input_size=input_size, hidden_size=hidden_size, num_layers=base_layer, bidirectional=True, dropout=dropout)
            self.rnn_parties = nn.GRU(input_size=input_size, hidden_size=hidden_size, num_layers=base_layer, bidirectional=True, dropout=dropout)
        elif self.base_model == 'Linear':
            self.base_linear = nn.Linear(input_size, 2 * hidden_size)
        else:
            print('Base model must be one of LSTM/GRU/Linear')
            raise NotImplementedError

        self.cognition_net = CognitionNetwork(n_features=2 * hidden_size, n_classes=n_classes, dropout=dropout, cuda_flag=cuda_flag, reason_steps=reason_steps)
        print(self)

    def forward(self, U, qmask, seq_lengths):
        U_s, U_p = None, None

        if self.base_model == 'LSTM':
            # (b,l,h), (b,l,p)
            U_, qmask_ = U.transpose(0, 1), qmask.transpose(0, 1)
            U_p_ = torch.zeros(U_.size()[0], U_.size()[1], self.hidden_size * 2).type(U.type())
            U_parties_ = [torch.zeros_like(U_).type(U_.type()) for _ in range(self.n_speakers)]
            for b in range(U_.size(0)):
                for p in range(len(U_parties_)):
                    index_i = torch.nonzero(qmask_[b][:, p]).squeeze(-1)
                    if index_i.size(0) > 0:
                        U_parties_[p][b][:index_i.size(0)] = U_[b][index_i]
            E_parties_ = [self.rnn_parties(U_parties_[p].transpose(0, 1))[0].transpose(0, 1) for p in range(len(U_parties_))]

            for b in range(U_p_.size(0)):
                for p in range(len(U_parties_)):
                    index_i = torch.nonzero(qmask_[b][:, p]).squeeze(-1)
                    if index_i.size(0) > 0: U_p_[b][index_i] = E_parties_[p][b][:index_i.size(0)]
            U_p = U_p_.transpose(0, 1)

            # (l,b,2*h) [(2*bi,b,h) * 2]
            U_s, hidden = self.rnn(U)

        elif self.base_model == 'GRU':
            U_, qmask_ = U.transpose(0, 1), qmask.transpose(0, 1)
            U_p_ = torch.zeros(U_.size()[0], U_.size()[1], self.hidden_size * 2).type(U.type())
            U_parties_ = [torch.zeros_like(U_).type(U_.type()) for _ in range(self.n_speakers)]  # default 2
            for b in range(U_.size(0)):
                for p in range(len(U_parties_)):
                    index_i = torch.nonzero(qmask_[b][:, p]).squeeze(-1)
                    if index_i.size(0) > 0: U_parties_[p][b][:index_i.size(0)] = U_[b][index_i]
            E_parties_ = [self.rnn_parties(U_parties_[p].transpose(0, 1))[0].transpose(0, 1) for p in range(len(U_parties_))]

            for b in range(U_p_.size(0)):
                for p in range(len(U_parties_)):
                    index_i = torch.nonzero(qmask_[b][:, p]).squeeze(-1)
                    if index_i.size(0) > 0: U_p_[b][index_i] = E_parties_[p][b][:index_i.size(0)]
            U_p = U_p_.transpose(0, 1)
            U_s, hidden = self.rnn(U)
        elif self.base_model == 'Linear':
            # TODO
            U = self.base_linear(U)
            U = self.dropout(F.relu(U))
            hidden = self.smax_fc(U)
            log_prob = F.log_softmax(hidden, 2)
            logits = torch.cat([log_prob[:, j, :][:seq_lengths[j]] for j in range(len(seq_lengths))])
            return logits

        logits = self.cognition_net(U_s, U_p, seq_lengths)
        return logits


#Train

In [14]:
# def get_train_valid_sampler(trainset, valid=0.1):
#     size = len(trainset)
#     idx = list(range(size))
#     split = int(valid * size)
#     return SubsetRandomSampler(idx[split:]), SubsetRandomSampler(idx[:split])

In [15]:
def get_ERC_loaders(path, n_classes, batch_size=32, valid=0.1, num_workers=0, pin_memory=False):
    trainset = ERCDataset(path=path, n_classes=n_classes, train=True, dev=False)
    devset = ERCDataset(path=path, n_classes=n_classes, train=False, dev=True)
    testset = ERCDataset(path=path, n_classes=n_classes, train=False, dev=False)
    
    train_loader = DataLoader(trainset,
                              batch_size=batch_size,
                              collate_fn=trainset.collate_fn,
                              num_workers=num_workers,
                              pin_memory=pin_memory)
    valid_loader = DataLoader(devset,
                              batch_size=batch_size,
                              collate_fn=trainset.collate_fn,
                              num_workers=num_workers,
                              pin_memory=pin_memory)

    test_loader = DataLoader(testset,
                             batch_size=batch_size,
                             collate_fn=testset.collate_fn,
                             num_workers=num_workers,
                             pin_memory=pin_memory)

    return train_loader, valid_loader, test_loader

In [16]:
# get_ERC_loaders("ERC_dataset_clean.pkl", 9)

In [17]:
def train_or_eval_model(model, loss_f, dataloader, epoch=0, train_flag=False, optimizer=None, cuda_flag=False, target_names=None,
                        tensorboard=False):
    assert not train_flag or optimizer != None
    losses, preds, labels, masks = [], [], [], []

    if train_flag:
        model.train()
    else:
        model.eval()

    for data in dataloader:
        if train_flag: optimizer.zero_grad()

        textf, qmask, umask, label = [d.cuda() for d in data[:-1]] if cuda_flag else data[:-1]
        seq_lengths = [(umask[j] == 1).nonzero().tolist()[-1][0] + 1 for j in range(len(umask))]

        dataf = textf 

        log_prob = model(dataf, qmask, seq_lengths)
        label = torch.cat([label[j][:seq_lengths[j]] for j in range(len(label))])
        umask = torch.cat([umask[j][:seq_lengths[j]] for j in range(len(umask))])
        loss = loss_f(log_prob, label)

        preds.append(torch.argmax(log_prob, 1).data.cpu().numpy())
        labels.append(label.data.cpu().numpy())
        masks.append(umask.view(-1).cpu().numpy())

        losses.append(loss.item())

        if train_flag:
            loss.backward()
            if tensorboard:
                for param in model.named_parameters():
                    writer.add_histogram(param[0], param[1].grad, epoch)
            optimizer.step()

    if preds != []:
        preds = np.concatenate(preds)
        labels = np.concatenate(labels)
    else:
        return float('nan'), float('nan'), float('nan'), [], []

    labels = np.array(labels)
    preds = np.array(preds)
    avg_loss = round(np.sum(losses) / len(losses), 4)
    avg_accuracy = round(accuracy_score(labels, preds) * 100, 2)
    avg_fscore = round(f1_score(labels, preds, average='weighted') * 100, 2)

    all_matrix = []
    all_matrix.append(
        metrics.classification_report(labels, preds, target_names=target_names if target_names else None, digits=4))
    all_matrix.append(["ACC"])
    for i in range(len(target_names)):
        all_matrix[-1].append("{}: {:.4f}".format(target_names[i], accuracy_score(labels[labels == i], preds[labels == i])))

    return avg_loss, avg_accuracy, avg_fscore, all_matrix, []


# Main

In [18]:
if __name__ == '__main__':

    parser = argparse.ArgumentParser()

    parser.add_argument('--status', type=str, default='train', help='optional status: train/test')

    parser.add_argument('--data_dir', type=str, default=f'Data_test_{no_label_data}.pkl', help='dataset dir')

    parser.add_argument('--output_dir', type=str, default='Model_Result/DialogueCRN', help='saved model dir')

    parser.add_argument('--load_model_state_dir', type=str, default='Model_Result/DialogueCRN/dialoguecrn_22.pkl', help='load model state dir')

    parser.add_argument('--base_model', default='LSTM', help='base model, LSTM/GRU/Linear')

    parser.add_argument('--base_layer', type=int, default=1, help='the number of base model layers, 1/2')

    parser.add_argument('--epochs', type=int, default=100, metavar='E', help='number of epochs')

    parser.add_argument('--patience', type=int, default=20, help='early stop')

    parser.add_argument('--batch-size', type=int, default=32, metavar='BS', help='batch size')

#     parser.add_argument('--valid_rate', type=float, default=0, metavar='valid_rate', help='valid rate: 0.0/0.1')

    parser.add_argument('--lr', type=float, default=0.0001, metavar='LR', help='learning rate')

    parser.add_argument('--l2', type=float, default=0.0002, metavar='L2', help='L2 regularization weight')

    parser.add_argument('--dropout', type=float, default=0.2, metavar='dropout', help='dropout rate')

    parser.add_argument('--step_s', type=int, default=2, help='the number of reason turns at situation-level,3')

    parser.add_argument('--step_p', type=int, default=0, help='the number of reason turns at speaker-level,0')

    parser.add_argument('--gamma', type=float, default=1, help='gamma 0/0.5/1/2')

    parser.add_argument('--no-cuda', action='store_true', default=False, help='does not use GPU')

    parser.add_argument('--class-weight', action='store_true', default=False, help='use class weights')

    parser.add_argument('--tensorboard', action='store_true', default=False, help='Enables tensorboard log')

    parser.add_argument('--cls_type', type=str, default='emotion', help='choose between sentiment or emotion')

    parser.add_argument('--seed', type=int, default=2021, help='random seed')

    args, unknown = parser.parse_known_args()
    print(args)

    epochs, batch_size, status, output_path, data_path, load_model_state_dir, base_model, base_layer  = \
        args.epochs, args.batch_size, args.status, args.output_dir, args.data_dir, args.load_model_state_dir, args.base_model, args.base_layer
    cuda_flag = torch.cuda.is_available() and not args.no_cuda
    reason_steps = [args.step_s, args.step_p]

    if args.tensorboard:
        from tensorboardX import SummaryWriter

        writer = SummaryWriter()

    # ERC dataset PhoBert large - embedding
    n_speakers, hidden_size, input_size = 29, 100, 1024
    n_classes = no_label_data
    target_names = list(set(df['Emotion']))
    class_weights = torch.FloatTensor(
            ERCDataset(data_path).calculate_weight())

    seed_everything(seed=args.seed)
    model = DialogueCRN(base_model=base_model,
                        base_layer=base_layer,
                        input_size=input_size,
                        hidden_size=hidden_size,
                        n_speakers=n_speakers,
                        n_classes=n_classes,
                        dropout=args.dropout,
                        cuda_flag=cuda_flag,
                        reason_steps=reason_steps)

    if cuda_flag:
        print('Running on GPU')
        # torch.cuda.set_device(0)
        class_weights = class_weights.cuda()
        model.cuda()
    else:
        print('Running on CPU')

    name = 'DialogueCRN'
    print('{} with {} as base model.'.format(name, base_model))
    print("The model have {} paramerters in total".format(sum(x.numel() for x in model.parameters())))

    loss_f = FocalLoss(gamma=args.gamma, alpha=class_weights if args.class_weight else None)
    optimizer = optim.Adam(model.parameters(), lr=args.lr, weight_decay=args.l2)
    train_loader, valid_loader, test_loader = get_ERC_loaders(data_path,
                                                               n_classes,
                                                               batch_size=batch_size,
                                                               num_workers=0)

    if status == 'train':
        all_test_fscore, all_test_acc = [], []
        best_epoch, best_epoch2, patience, best_eval_fscore, best_eval_loss = -1, -1, 0, 0, None
        patience2 = 0
        for e in range(epochs):
            start_time = time.time()
            train_loss, train_acc, train_fscore, _, _ = train_or_eval_model(model=model, loss_f=loss_f, dataloader=train_loader, train_flag=True,
                                                                            optimizer=optimizer, cuda_flag=cuda_flag,
                                                                            target_names=target_names)
            valid_loss, valid_acc, valid_fscore, _, _ = train_or_eval_model(model=model, loss_f=loss_f, dataloader=valid_loader,
                                                                            cuda_flag=cuda_flag, target_names=target_names)
            test_loss, test_acc, test_fscore, test_metrics, _ = train_or_eval_model(model=model, loss_f=loss_f, dataloader=test_loader,
                                                                                    cuda_flag=cuda_flag, target_names=target_names)
            all_test_fscore.append(test_fscore)
            all_test_acc.append(test_acc)


            eval_loss, _, eval_fscore = valid_loss, valid_acc, valid_fscore
            
            if e == 0 or best_eval_fscore < eval_fscore:
                patience = 0
                best_epoch, best_eval_fscore = e, eval_fscore
                if not os.path.exists(output_path): os.makedirs(output_path)
                save_model_dir = os.path.join(output_path, '{}_{}.pkl'.format(name, e).lower())
                torch.save(model.state_dict(), save_model_dir)
            else:
                patience += 1
            if best_eval_loss is None:
                best_eval_loss = eval_loss
                best_epoch2 = 0
            else:
                if eval_loss < best_eval_loss:
                    best_epoch2, best_eval_loss = e, eval_loss
                    patience2 = 0
                    if not os.path.exists(output_path): os.makedirs(output_path)
                    save_model_dir = os.path.join(output_path, 'loss_{}_{}.pkl'.format(name, e).lower())
                    torch.save(model.state_dict(), save_model_dir)

                else:
                    patience2 += 1

            if args.tensorboard:
                writer.add_scalar('train: accuracy/f1/loss', train_acc / train_fscore / train_loss, e)
                writer.add_scalar('valid: accuracy/f1/loss', valid_acc / valid_fscore / valid_loss, e)
                writer.add_scalar('test: accuracy/f1/loss', test_acc / test_fscore / test_loss, e)
                writer.close()

            print(
                'epoch: {}, train_loss: {}, train_acc: {}, train_fscore: {}, valid_loss: {}, valid_acc: {}, valid_fscore: {}, test_loss: {}, test_acc: {}, test_fscore: {}, time: {} sec'. \
                    format(e, train_loss, train_acc, train_fscore, valid_loss, valid_acc, valid_fscore, test_loss, test_acc, test_fscore,
                           round(time.time() - start_time, 2)))
            print(test_metrics[0])
            print(test_metrics[1])
            print('\n')

            if patience >= args.patience and patience2 >= args.patience:
                print('Early stoping...', patience, patience2)
                break

        print('Final Test performance...')
        print('Early stoping...', patience, patience2)
        print('Eval-metric: F1, Epoch: {}, best_eval_fscore: {}, Accuracy: {}, F1-Score: {}'.format(best_epoch, best_eval_fscore,
                                                                                                    all_test_acc[best_epoch] if best_epoch >= 0 else 0,
                                                                                                    all_test_fscore[best_epoch] if best_epoch >= 0 else 0))
        print('Eval-metric: Loss, Epoch: {}, Accuracy: {}, F1-Score: {}'.format(best_epoch2,
                                                                                all_test_acc[best_epoch2] if best_epoch2 >= 0 else 0,
                                                                                all_test_fscore[best_epoch2] if best_epoch2 >= 0 else 0))

    elif status == 'test':
        start_time = time.time()
        model.load_state_dict(torch.load(args.load_model_state_dir))
        test_loss, test_acc, test_fscore, test_metrics, test_outputs = train_or_eval_model(model=model, loss_f=loss_f, dataloader=test_loader,
                                                                                           cuda_flag=cuda_flag,
                                                                                           target_names=target_names)

        if args.tensorboard:
            writer.add_scalar('test: accuracy/loss', test_acc / test_loss, 0)
            writer.close()
        print('test_loss: {}, test_acc: {}, test_fscore: {}, time: {} sec'.format(test_loss, test_acc, test_fscore, round(time.time() - start_time, 2)))
        print(test_metrics[0])
        print(test_metrics[1])
    else:
        print('the status must be one of train/test')
        exit(0)


Namespace(base_layer=1, base_model='LSTM', batch_size=32, class_weight=False, cls_type='emotion', data_dir='Data_test_9_2.pkl', dropout=0.2, epochs=100, gamma=1, l2=0.0002, load_model_state_dir='Model_Result/DialogueCRN/dialoguecrn_22.pkl', lr=0.0001, no_cuda=False, output_dir='Model_Result/DialogueCRN', patience=20, seed=2021, status='train', step_p=0, step_s=2, tensorboard=False)
2021
DialogueCRN(
  (rnn): LSTM(1024, 100, dropout=0.2, bidirectional=True)
  (rnn_parties): LSTM(1024, 100, dropout=0.2, bidirectional=True)
  (cognition_net): CognitionNetwork(
    (fc): Linear(in_features=200, out_features=400, bias=True)
    (reason_modules): ModuleList(
      (0): ReasonModule(200, 400)
      (1): ReasonModule(200, 400)
    )
    (dropout): Dropout(p=0.2, inplace=False)
    (smax_fc): Linear(in_features=800, out_features=9, bias=True)
  )
)
Running on GPU
DialogueCRN with LSTM as base model.
The model have 2370809 paramerters in total
epoch: 0, train_loss: 1.2261, train_acc: 54.77, trai

epoch: 7, train_loss: 0.3109, train_acc: 85.28, train_fscore: 84.99, valid_loss: 1.2084, valid_acc: 51.87, valid_fscore: 51.54, test_loss: 1.1914, test_acc: 50.13, test_fscore: 49.67, time: 27.94 sec
              precision    recall  f1-score   support

        Fear     0.5154    0.5590    0.5363       390
 Frustration     0.5688    0.6253    0.5957       443
     Sadness     0.3917    0.3507    0.3701       268
     Neutral     0.6321    0.4891    0.5514       137
     Disgust     0.3182    0.2188    0.2593        32
     Excited     0.3504    0.3661    0.3581       112
         Joy     0.6087    0.5000    0.5490        28
    Surprise     0.4533    0.4626    0.4579       147
       Anger     0.0000    0.0000    0.0000        11

    accuracy                         0.5013      1568
   macro avg     0.4265    0.3968    0.4086      1568
weighted avg     0.4959    0.5013    0.4967      1568

['ACC', 'Fear: 0.5590', 'Frustration: 0.6253', 'Sadness: 0.3507', 'Neutral: 0.4891', 'Disgust: 

epoch: 15, train_loss: 0.2354, train_acc: 88.15, train_fscore: 87.95, valid_loss: 1.2844, valid_acc: 53.14, valid_fscore: 53.09, test_loss: 1.2543, test_acc: 52.81, test_fscore: 52.58, time: 29.31 sec
              precision    recall  f1-score   support

        Fear     0.5268    0.5795    0.5519       390
 Frustration     0.6130    0.6185    0.6157       443
     Sadness     0.4555    0.4776    0.4663       268
     Neutral     0.6286    0.4818    0.5455       137
     Disgust     0.3333    0.2500    0.2857        32
     Excited     0.3333    0.3393    0.3363       112
         Joy     0.6522    0.5357    0.5882        28
    Surprise     0.5034    0.4966    0.5000       147
       Anger     0.0000    0.0000    0.0000        11

    accuracy                         0.5281      1568
   macro avg     0.4496    0.4199    0.4322      1568
weighted avg     0.5264    0.5281    0.5258      1568

['ACC', 'Fear: 0.5795', 'Frustration: 0.6185', 'Sadness: 0.4776', 'Neutral: 0.4818', 'Disgust:

epoch: 23, train_loss: 0.1669, train_acc: 90.86, train_fscore: 90.71, valid_loss: 1.4526, valid_acc: 52.58, valid_fscore: 52.58, test_loss: 1.4213, test_acc: 52.81, test_fscore: 52.55, time: 28.55 sec
              precision    recall  f1-score   support

        Fear     0.5293    0.5795    0.5532       390
 Frustration     0.6115    0.6253    0.6183       443
     Sadness     0.4539    0.4776    0.4655       268
     Neutral     0.6286    0.4818    0.5455       137
     Disgust     0.3158    0.1875    0.2353        32
     Excited     0.3306    0.3661    0.3475       112
         Joy     0.6522    0.5357    0.5882        28
    Surprise     0.5111    0.4694    0.4894       147
       Anger     0.0000    0.0000    0.0000        11

    accuracy                         0.5281      1568
   macro avg     0.4481    0.4136    0.4270      1568
weighted avg     0.5265    0.5281    0.5255      1568

['ACC', 'Fear: 0.5795', 'Frustration: 0.6253', 'Sadness: 0.4776', 'Neutral: 0.4818', 'Disgust:

epoch: 31, train_loss: 0.1239, train_acc: 92.88, train_fscore: 92.82, valid_loss: 1.6382, valid_acc: 52.16, valid_fscore: 51.49, test_loss: 1.6096, test_acc: 52.04, test_fscore: 51.17, time: 29.28 sec
              precision    recall  f1-score   support

        Fear     0.5022    0.5897    0.5425       390
 Frustration     0.5758    0.6862    0.6262       443
     Sadness     0.4587    0.3731    0.4115       268
     Neutral     0.6465    0.4672    0.5424       137
     Disgust     0.3529    0.1875    0.2449        32
     Excited     0.3366    0.3036    0.3192       112
         Joy     0.6818    0.5357    0.6000        28
    Surprise     0.5081    0.4286    0.4649       147
       Anger     0.0000    0.0000    0.0000        11

    accuracy                         0.5204      1568
   macro avg     0.4514    0.3968    0.4168      1568
weighted avg     0.5135    0.5204    0.5117      1568

['ACC', 'Fear: 0.5897', 'Frustration: 0.6862', 'Sadness: 0.3731', 'Neutral: 0.4672', 'Disgust: